

# CNN Implementation

Importing libraries

In [ ]:
import pandas as pd
import numpy as np
from numpy import array
from keras import layers
from datetime import datetime
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Embedding,Dense
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from nltk.tokenize import word_tokenize
from keras.layers.core import Dense, Dropout, Activation, Flatten, Reshape
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tensorflow.keras.optimizers import SGD
from keras.layers import LSTM, SimpleRNN, GRU, Bidirectional, BatchNormalization,Convolution1D,MaxPooling1D, Reshape, GlobalAveragePooling1D
import h5py
from keras import callbacks
from sklearn import metrics

Upload Data

In [ ]:
from google.colab import files
dataset = files.upload()

Saving D1_all_shuffled.csv to D1_all_shuffled.csv


Loading Data

In [ ]:
df = pd.read_csv(r"D1_all_shuffled.csv")
X = df.iloc[:,df.columns!= 'label']
y = df.iloc[:,-1]

print(X)
print(y)

       digitsHostname  PresentAtrateHost  AvgWordLenHost  LongWordLenHost  \
0                   0                  0       10.500000               18   
1                   0                  0        5.000000                9   
2                   5                  0        8.333333               13   
3                   0                  0        5.000000                7   
4                   0                  0        7.666667               15   
...               ...                ...             ...              ...   
83852               0                  0        4.000000                5   
83853               0                  0        5.666667               11   
83854               0                  0        4.333333                7   
83855               2                  0        3.333333                4   
83856               0                  0        5.000000                9   

       HostNameLen  domainLen  digitsDomain  dotsHost  HypensHost  \
0     

In [ ]:
print(X.iloc[0])

digitsHostname        0.000000
PresentAtrateHost     0.000000
AvgWordLenHost       10.500000
LongWordLenHost      18.000000
HostNameLen          22.000000
                       ...    
]                     0.000000
-                     0.000000
_                     0.052632
.                     0.105263
~                     0.000000
Name: 0, Length: 93, dtype: float64


Defining Keras Models

In [ ]:
model=Sequential([
  
    Convolution1D(64,kernel_size=3,padding="same",activation="relu",input_shape=(93,1)),
    MaxPooling1D(pool_size=2),
    BatchNormalization(),
    LSTM(64, return_sequences=False),
    Reshape((64, 1), input_shape = (93, )),
    Dropout(0.5),

    Convolution1D(128,kernel_size=128,padding="same",activation="relu"),
    MaxPooling1D(pool_size=2),
    BatchNormalization(),
    LSTM(128, return_sequences=False),
    Reshape((128, 1), input_shape = (128, )),
    Dropout(0.5),

    Convolution1D(256,kernel_size=256,padding="same",activation="relu"),
    MaxPooling1D(pool_size=2),
    BatchNormalization(),
    LSTM(256, return_sequences=False),
    Reshape((256, 1), input_shape = (256, )),
    Dropout(0.5),


    Convolution1D(512,kernel_size=512,padding="same",activation="relu"),
    GlobalAveragePooling1D(),
    Dropout(0.5),
    Dense(1,activation='sigmoid')

])

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
for layer in model.layers:
    print(layer.output_shape)

(None, 93, 32)
(None, 46, 32)
(None, 46, 32)
(None, 32)
(None, 32, 1)
(None, 32, 1)
(None, 32, 64)
(None, 16, 64)
(None, 16, 64)
(None, 64)
(None, 64, 1)
(None, 64, 1)
(None, 64, 128)
(None, 32, 128)
(None, 32, 128)
(None, 128)
(None, 128, 1)
(None, 128, 1)
(None, 128, 256)
(None, 64, 256)
(None, 64, 256)
(None, 256)
(None, 256, 1)
(None, 256, 1)
(None, 256, 512)
(None, 512)
(None, 512)
(None, 1)


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 93, 32)            128       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 46, 32)           0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 46, 32)           128       
 ormalization)                                                   
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 reshape (Reshape)           (None, 32, 1)             0         
                                                                 
 dropout (Dropout)           (None, 32, 1)             0

In [ ]:
def evaluate_model(X_test,y_test):
    loss, acc = model.evaluate(X_test,y_test)
    return acc

In [ ]:
kf = KFold(n_splits=5)
accuracies = []

start = datetime.now()
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    print(X_train)
    y_train, y_test = y[train_index], y[test_index]
    y_train = y_train.values.reshape(-1,1)
    y_test  = y_test.values.reshape(-1,1)
    print(y_train)
    model.fit(X_train,y_train,batch_size = 64,epochs=10)
    # model.save("hybridmodel")
    accuracy = evaluate_model(X_test,y_test)
    print(accuracy)
    accuracies.append(accuracy)
end = datetime.now()
td = (end - start).total_seconds()
print("Total Execution Time: ",td)

TRAIN: [16772 16773 16774 ... 83854 83855 83856] TEST: [    0     1     2 ... 16769 16770 16771]
       digitsHostname  PresentAtrateHost  AvgWordLenHost  LongWordLenHost  \
16772               0                  0        5.000000                7   
16773               0                  0        8.500000               14   
16774               0                  0        8.500000               15   
16775               0                  0        4.400000                7   
16776               0                  0        4.333333                6   
...               ...                ...             ...              ...   
83852               0                  0        4.000000                5   
83853               0                  0        5.666667               11   
83854               0                  0        4.333333                7   
83855               2                  0        3.333333                4   
83856               0                  0        5.000000

KeyboardInterrupt: ignored

In [ ]:
print(accuracies)

[0.9323276877403259, 0.9305986166000366, 0.9425198435783386, 0.9440701007843018, 0.9459185600280762]


In [ ]:
print(sum(accuracies)/len(accuracies))

0.9390869617462159


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model.save(r'C:\Users\hp\Desktop\test')